In [2]:
import time
import hashlib

import numpy as np
import pandas as pd

In [9]:
# data = pd.read_csv("data/data.csv")
data = pd.read_csv("data/society/cdhit80_data_seq.csv")

In [15]:
len(data)

3998

In [4]:
def k_mers(data, n):
    return [[s[i:i + n].lower() for i in range(len(s) - n)] for s in data]

In [5]:
from collections import Counter

def build_vocab(data):
    word_counts = Counter(row.lower() for sample in data for row in sample)
    vocab = [w for w, f in iter(word_counts.items())]
    return vocab

In [11]:
kmer_array = k_mers(data['Sequence'], 2)
vocab = build_vocab(kmer_array)

In [12]:
row_count = [[Counter(row)[key] for key in vocab] for row in kmer_array ]

In [14]:
from scipy import stats

row_count = np.array(row_count)
print(row_count)
z = stats.zscore(row_count, axis=1, ddof=1)
print(row_count.shape)
print(z.shape)

[[7484 5884 4306 ... 6343 5368 5271]
 [2970 2204 1449 ... 2501 3457 2516]
 [2302 2323 1042 ... 1991  876  941]
 ...
 [   3    2    1 ...    5   10    4]
 [   7    2    4 ...    4    2    3]
 [   4    3    9 ...    4    0    1]]
(3998, 16)
(3998, 16)
